# **Implémentation from scratch de Word2Vec**

Ce notebook implémente les modèles CBOW et Skip-gram avec des optimisations comme l'échantillonnage négatif et le sous-échantillonnage des mots fréquents.

## **1. Introduction et fondements théoriques**

- Explication des concepts fondamentaux de représentation vectorielle des mots
- Principes mathématiques sous-jacents (calcul de probabilités, descente de gradient, etc.)

## **2. Préparation des données**

Cette section couvre le prétraitement des données, la tokenisation, la construction du vocabulaire et la génération des paires d'apprentissage.

In [1]:
# Importation des bibliothèques nécessaires
import numpy as np
from collections import Counter
import random
import pandas as pd

# Chargement du corpus depuis le fichier IMDB Dataset.csv
data_path = '/Volumes/SSD_crucial/Cours/master/info804/projet/data/IMDB Dataset.csv'
df = pd.read_csv(data_path)

# Prétraitement du corpus
def preprocess_text(text):
    return text.lower().split()

corpus = []
for review in df['review'][:1000]:  # Limitation à 1000 reviews pour des raisons de performance
    corpus.extend(preprocess_text(review))

# Construction du vocabulaire
vocab = set(corpus)
word_to_index = {word: idx for idx, word in enumerate(vocab)}
index_to_word = {idx: word for word, idx in word_to_index.items()}

# Génération des paires contexte-cible pour CBOW et Skip-gram
def generate_training_data(corpus, window_size=2):
    training_data = []
    for i, word in enumerate(corpus):
        target_word = word_to_index[word]
        context = []
        for j in range(-window_size, window_size + 1):
            if j != 0 and 0 <= i + j < len(corpus):
                context.append(word_to_index[corpus[i + j]])
        training_data.append((context, target_word))
    return training_data

training_data = generate_training_data(corpus)
print("Exemple de données d'entraînement:", training_data[:5])

Exemple de données d'entraînement: [([13187, 29537], 7456), ([7456, 29537, 13241], 13187), ([7456, 13187, 13241, 7925], 29537), ([13187, 29537, 7925, 7538], 13241), ([29537, 13241, 7538, 14523], 7925)]


## **3. Implémentation du modèle CBOW**

Cette section implémente le modèle CBOW de base.

In [2]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Embedding, Lambda
import tensorflow.keras.backend as K

# Modèle CBOW avec ANN
class CBOW:
    def __init__(self, vocab_size, embedding_dim):
        self.vocab_size = vocab_size
        self.embedding_dim = embedding_dim

        # Définition du modèle ANN
        context_input = Input(shape=(None,), name="context_input")
        embedding = Embedding(input_dim=vocab_size, output_dim=embedding_dim, name="embedding")(context_input)
        mean_embedding = Lambda(lambda x: K.mean(x, axis=1), name="mean_embedding")(embedding)
        output = Dense(vocab_size, activation="softmax", name="output")(mean_embedding)

        self.model = Model(inputs=context_input, outputs=output)
        self.model.compile(optimizer="adam", loss="sparse_categorical_crossentropy")

    def train(self, contexts, targets, epochs=10, batch_size=32):
        self.model.fit(contexts, targets, epochs=epochs, batch_size=batch_size)

# Initialisation du modèle CBOW
cbow_model = CBOW(vocab_size=len(vocab), embedding_dim=10)
print("Modèle CBOW avec ANN initialisé.")

Modèle CBOW avec ANN initialisé.


## **4. Implémentation du modèle Skip-gram**

Cette section implémente le modèle Skip-gram.

In [3]:
# Modèle Skip-gram avec ANN
class SkipGram:
    def __init__(self, vocab_size, embedding_dim):
        self.vocab_size = vocab_size
        self.embedding_dim = embedding_dim

        # Définition du modèle ANN
        target_input = Input(shape=(1,), name="target_input")
        context_input = Input(shape=(1,), name="context_input")

        embedding = Embedding(input_dim=vocab_size, output_dim=embedding_dim, name="embedding")
        target_embedding = embedding(target_input)
        context_embedding = embedding(context_input)

        dot_product = Lambda(lambda x: K.sum(x[0] * x[1], axis=-1), name="dot_product")([target_embedding, context_embedding])
        output = Dense(1, activation="sigmoid", name="output")(dot_product)

        self.model = Model(inputs=[target_input, context_input], outputs=output)
        self.model.compile(optimizer="adam", loss="binary_crossentropy")

    def train(self, targets, contexts, labels, epochs=10, batch_size=32):
        self.model.fit([targets, contexts], labels, epochs=epochs, batch_size=batch_size)

# Initialisation du modèle Skip-gram
skipgram_model = SkipGram(vocab_size=len(vocab), embedding_dim=10)
print("Modèle Skip-gram avec ANN initialisé.")

Modèle Skip-gram avec ANN initialisé.


## **5. Optimisations**

### 5.1 Échantillonnage négatif

In [4]:
def negative_sampling(vocab_size, target_word, num_samples=5):
    negative_samples = []
    while len(negative_samples) < num_samples:
        sample = random.randint(0, vocab_size - 1)
        if sample != target_word:
            negative_samples.append(sample)
    return negative_samples

# Exemple d'échantillonnage négatif
print("Exemple d'échantillons négatifs:", negative_sampling(len(vocab), target_word=2))

Exemple d'échantillons négatifs: [28327, 4152, 30559, 29907, 9900]


### 5.2 Sous-échantillonnage des mots fréquents

In [5]:
def subsample_frequent_words(corpus, threshold=1e-5):
    word_counts = Counter(corpus)
    total_count = len(corpus)
    subsampled_corpus = []
    for word in corpus:
        freq = word_counts[word] / total_count
        prob = (np.sqrt(freq / threshold) + 1) * (threshold / freq)
        if random.random() < prob:
            subsampled_corpus.append(word)
    return subsampled_corpus

# Exemple de sous-échantillonnage
subsampled_corpus = subsample_frequent_words(corpus)
print("Corpus après sous-échantillonnage:", subsampled_corpus)

Corpus après sous-échantillonnage: ['mentioned', '1', 'oz', 'episode', "you'll", 'hooked.', 'right,', 'happened', 'me.<br', 'struck', 'oz', 'brutality', 'unflinching', 'violence,', 'which', 'go.', 'trust', 'faint', 'hearted', 'timid.', 'pulls', 'punches', 'regards', 'drugs,', 'sex', 'violence.', 'its', 'hardcore,', 'word.<br', '/>it', 'oz', 'nickname', 'given', 'oswald', 'maximum', 'security', 'state', 'penitentary.', 'focuses', 'mainly', 'emerald', 'city,', 'experimental', 'section', 'prison', 'cells', 'glass', 'fronts', 'inwards,', 'privacy', 'high', 'agenda.', 'em', 'home', 'many..aryans,', 'muslims,', 'gangstas,', 'latinos,', 'christians,', 'italians,', 'irish', 'and', 'more....so', 'scuffles,', 'death', 'stares,', 'dodgy', 'dealings', 'shady', 'agreements', 'away.<br', 'main', 'appeal', "wouldn't", 'dare.', 'pictures', 'painted', 'mainstream', 'audiences,', 'charm,', 'romance...oz', 'mess', 'around.', 'struck', 'surreal,', "couldn't", 'say', 'but', 'watched', 'more,', 'developed',

## **6. Visualisation et évaluation**

Cette section visualise les embeddings et évalue leur qualité.

In [6]:
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
from tqdm import tqdm  # Ajout de tqdm

def extract_embeddings(model):
    # Extraction des poids de la couche embedding
    return model.model.get_layer('embedding').get_weights()[0]

def visualize_embeddings(embeddings, word_to_index):
    pca = PCA(n_components=2)
    reduced_embeddings = pca.fit_transform(embeddings)
    plt.figure(figsize=(10, 10))
    
    # Boucle avec barre de progression
    for word, idx in tqdm(word_to_index.items(), desc="Annotation des mots"):
        plt.scatter(reduced_embeddings[idx, 0], reduced_embeddings[idx, 1])
        plt.annotate(word, (reduced_embeddings[idx, 0], reduced_embeddings[idx, 1]))
    
    plt.show()

# Visualisation des embeddings (exemple)
cbow_embeddings = extract_embeddings(cbow_model)
visualize_embeddings(cbow_embeddings, word_to_index)


Annotation des mots: 100%|██████████| 32162/32162 [41:50<00:00, 12.81it/s]  
/Volumes/SSD_crucial/Cours/master/info804/projet/.venv/lib/python3.11/site-packages/IPython/core/pylabtools.py:170: UserWarning: Glyph 151 (\x97) missing from font(s) DejaVu Sans.
  fig.canvas.print_figure(bytes_io, **kw)
/Volumes/SSD_crucial/Cours/master/info804/projet/.venv/lib/python3.11/site-packages/IPython/core/pylabtools.py:170: UserWarning: Glyph 150 (\x96) missing from font(s) DejaVu Sans.
  fig.canvas.print_figure(bytes_io, **kw)
/Volumes/SSD_crucial/Cours/master/info804/projet/.venv/lib/python3.11/site-packages/IPython/core/pylabtools.py:170: UserWarning: Glyph 132 (\x84) missing from font(s) DejaVu Sans.
  fig.canvas.print_figure(bytes_io, **kw)


ValueError: 
"larda$$",
      ^
ParseException: Expected end of text, found '$'  (at char 6), (line:1, col:7)

<Figure size 1000x1000 with 1 Axes>